크롤링 사이트 : https://esg.krx.co.kr/contents/02/02030000/ESG02030000.jsp

1. 기업 별 지속가능경영보고서 접속링크 크롤링

In [2]:
import pandas as pd
import requests
import json

# 크롤링할 년도 범위 및 페이지 범위 설정
years = range(2013, 2024)
pages = range(1, 19)  # 예시로 1부터 18페이지까지 크롤링

# 결과를 저장할 리스트 초기화
data_df = []

# 년도와 페이지를 모두 반복하며 크롤링 수행
for year in years:
    for page in pages:
        url = 'https://esg.krx.co.kr/contents/99/ESG99000001.jspx'
        payload = {
            'sch_yy': year,
            'sch_tp': 'N', 
            'pagePath': '/contents/02/02030000/ESG02030000.jsp', 
            'code': '02/02030000/esg02030000_01', 
            'curPage': page
        }

        response = requests.post(url, data=payload, headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'})

        # 응답 확인
        if response.status_code == 200:
            python_data = json.loads(response.text)
            for ele in python_data['result']:
                data_df.append([ele['com_abbrv'], ele['yy'], ele['save_file_nm']])
        else:
            print(f"페이지 {page}, 년도 {year} 크롤링 실패")

# 데이터프레임 생성
columns = ['회사명', '발행년도', '코드']
df = pd.DataFrame(data_df, columns=columns)
df['페이지접속링크'] = 'https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno='+df['코드']+'&docno=&viewerhost=&viewerport='

# 코스피 200 기업명 리스트 가져오기
f = open('top200_name_list.csv','r',encoding='utf-8')
data = f.read().split('\n')

# 데이터프레임의 조직명과 코스피 200 리스트의 회사명을 비교하여 코스피 200에 속하는 행만 추출
kospi200_companies_df = df[df['회사명'].isin(data)]

2. pdf링크 크롤링

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# 웹 드라이버 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 새로운 열 생성 및 초기화
kospi200_companies_df['pdf다운로드링크'] = ''

# 데이터프레임 반복문
for index, row in kospi200_companies_df.iterrows():
    # 웹사이트에 접속
    driver.get(row['페이지접속링크'])

    # 요소 클릭
    try:
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'attachedDoc'))).click()
    except:
        print("요소를 찾을 수 없습니다.")
        print(index, row)
        print(row['페이지접속링크'])
        continue

    # 두 번째 옵션 클릭
    try:
        options = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.XPATH, "//select[@id='attachedDoc']/option")))
        if len(options) > 1:
            options[1].click()  # 두 번째 옵션 선택
    except:
        print("옵션을 찾을 수 없습니다.")
        continue

    # iframe 전환
    try:
        driver.switch_to.frame('docViewFrm')
    except:
        print("iframe을 찾을 수 없습니다.")
        continue

    # PDF 다운로드 링크 가져오기
    try:
        pdf_download_link = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div/div/a"))).get_attribute('href')
        # 데이터프레임에 저장
        kospi200_companies_df.at[index, 'pdf다운로드링크'] = pdf_download_link
    except:
        print("PDF 다운로드 링크를 찾을 수 없습니다.")
        kospi200_companies_df.at[index, 'pdf다운로드링크'] = None

# 웹 드라이버 종료
driver.close()
driver.quit()

C:\Users\SDA05\AppData\Local\Temp\ipykernel_3200\1754232515.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kospi200_companies_df['pdf다운로드링크'] = ''


PDF 다운로드 링크를 찾을 수 없습니다.


3. 결측치

3-1. pdf 링크 추가로 작성

In [33]:
print(kospi200_companies_df.iloc[72])
company = 'DGB금융지주'
year = '2019'
content = 'https://kind.krx.co.kr/external/2019/07/25/000147/20190725000368/DGB%EA%B8%88%EC%9C%B5%EA%B7%B8%EB%A3%B9%202018-2019%20%EC%A7%80%EC%86%8D%EA%B0%80%EB%8A%A5%EA%B2%BD%EC%98%81%EB%B3%B4%EA%B3%A0%EC%84%9C.pdf'

kospi200_companies_df.loc[(kospi200_companies_df['회사명'] == company) & (kospi200_companies_df['발행년도'] == year), 'pdf다운로드링크'] = content
print(kospi200_companies_df.iloc[72])

회사명                                                    DGB금융지주
발행년도                                                      2019
코드                                              20190725000147
페이지접속링크      https://kind.krx.co.kr/common/disclsviewer.do?...
pdf다운로드링크                                                     
Name: 81, dtype: object
회사명                                                    DGB금융지주
발행년도                                                      2019
코드                                              20190725000147
페이지접속링크      https://kind.krx.co.kr/common/disclsviewer.do?...
pdf다운로드링크    https://kind.krx.co.kr/external/2019/07/25/000...
Name: 81, dtype: object


In [44]:
print(kospi200_companies_df.iloc[99])
company = '삼성전자'
year = '2020'
content = 'https://kind.krx.co.kr/external/2020/07/15/000250/20200715000308/%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90%20%EC%A7%80%EC%86%8D%EA%B0%80%EB%8A%A5%EA%B2%BD%EC%98%81%EB%B3%B4%EA%B3%A0%EC%84%9C%202020.pdf'

kospi200_companies_df.loc[(kospi200_companies_df['회사명'] == company) & (kospi200_companies_df['발행년도'] == year), 'pdf다운로드링크'] = content
print(kospi200_companies_df.iloc[99])

회사명                                                       삼성전자
발행년도                                                      2020
코드                                              20200715000250
페이지접속링크      https://kind.krx.co.kr/common/disclsviewer.do?...
pdf다운로드링크                                                     
Name: 111, dtype: object
회사명                                                       삼성전자
발행년도                                                      2020
코드                                              20200715000250
페이지접속링크      https://kind.krx.co.kr/common/disclsviewer.do?...
pdf다운로드링크    https://kind.krx.co.kr/external/2020/07/15/000...
Name: 111, dtype: object


In [45]:
print(kospi200_companies_df.iloc[179])
company = '삼성증권'
year = '2022'
content = 'https://kind.krx.co.kr/external/2022/08/26/000347/20220826000968/2021%20%EC%82%BC%EC%84%B1%EC%A6%9D%EA%B6%8C%20%ED%86%B5%ED%95%A9%EB%B3%B4%EA%B3%A0%EC%84%9C%28%EA%B5%AD%EB%AC%B8%29.pdf'

kospi200_companies_df.loc[(kospi200_companies_df['회사명'] == company) & (kospi200_companies_df['발행년도'] == year), 'pdf다운로드링크'] = content
print(kospi200_companies_df.iloc[179])

회사명                                                       삼성증권
발행년도                                                      2022
코드                                              20220826000347
페이지접속링크      https://kind.krx.co.kr/common/disclsviewer.do?...
pdf다운로드링크                                                     
Name: 222, dtype: object
회사명                                                       삼성증권
발행년도                                                      2022
코드                                              20220826000347
페이지접속링크      https://kind.krx.co.kr/common/disclsviewer.do?...
pdf다운로드링크    https://kind.krx.co.kr/external/2022/08/26/000...
Name: 222, dtype: object


3-2. pdf가 없는 경우는 제거

In [4]:
# 결측치인 행의 인덱스 반환
kospi200_companies_df['pdf다운로드링크'][kospi200_companies_df['pdf다운로드링크'].isnull()]

29    None
Name: pdf다운로드링크, dtype: object

In [5]:
kospi200_companies_df.iloc[29]#['페이지접속링크'] #-> pdf 파일 존재하지 않음 == 당연히 pdf 다운로드 링크 없음
# 결측치가 존재하는 행  제거
kospi200_companies_df = kospi200_companies_df.drop(29)

4. pdf 다운로드

In [6]:
# pdf 다운로드
import os
import urllib.request
import traceback

total_info = [tuple(row) for row in kospi200_companies_df.values] # 데이터프레임을 리스트 안의 튜플 데이터 형태로 변환

dir = './pdf/' # pdf 파일 저장할 경로 설정
if not os.path.exists(dir): # 폴더 없는 경우에 생성하는 코드
    os.makedirs(dir)

for i in range(len(total_info)):
    if i==0:
        print('pdf 다운로드 시작')
    try:
        file_name = total_info[i][0]+'_'+str(total_info[i][1])+'.pdf'
        urllib.request.urlretrieve(total_info[i][4], dir+file_name)
    except:
        print(f"error : {total_info[i][0]}")
        print(traceback.format_exc())
        i += 1 # 에러가 발생한 경우, 해당 항목부터 다시 크롤링할 수 있도록 i를 1 증가시킴
    if i!=0 and i%100==0:
        print(f'pdf 다운로드 진행률 : {i}/{len(total_info)}') # 100개 다운로드마다 알림
print('완료')

pdf 다운로드 시작
pdf 다운로드 진행률 : 100/362
pdf 다운로드 진행률 : 200/362
pdf 다운로드 진행률 : 300/362
완료


5. 데이터프레임에 pdf 내용 저장 (전처리X)

In [7]:
import pandas as pd
import os
import fitz
import kss
import pandas as pd

pdf_path = os.getcwd() + '\\pdf\\'

final_list = []
for idx, row in kospi200_companies_df.iterrows():
    pdf_file_path = pdf_path + row['회사명'] + '_' + str(row['발행년도']) + '.pdf'
    try:
        with fitz.open(pdf_file_path) as doc:
            contents = ''
            for page in doc:
                contents += page.get_text()
            sents = kss.split_sentences(contents)
            sents = [sent.replace('\n', '') for sent in sents]
            final_list.append(sents)

    except:
        print('해당파일 제외 :', pdf_file_path)
        # 파일을 열 수 없는 경우 None 값을 추가합니다.
        final_list.append(None)

# 데이터프레임을 복제하여 새로운 데이터프레임을 생성합니다.
new_df = kospi200_companies_df.copy()

# '내용' 열을 final_list의 각 항목으로 설정합니다.
new_df['내용'] = final_list

# 수정된 데이터프레임을 저장 및 출력합니다.
new_df.to_csv('지속가능경영보고서전처리전내용.csv',index=False)
new_df

[Kss]: Oh! You have mecab in your environment. Kss will take this as a backend! :D



해당파일 제외 : c:\Users\SDA05\Desktop\ESG_TEAM1\지속가능경영보고서_키워드추출용\sustainable_management_report_crawling\pdf\LG유플러스_2015.pdf
해당파일 제외 : c:\Users\SDA05\Desktop\ESG_TEAM1\지속가능경영보고서_키워드추출용\sustainable_management_report_crawling\pdf\한화시스템_2021.pdf


,회사명,발행년도,코드,페이지접속링크,pdf다운로드링크,내용
0,삼성물산,2013,20131224000500,https://kind.krx.co.kr/common/disclsviewer.do?...,https://kind.krx.co.kr/external/2013/12/24/000...,[Samsung C&T Corporation Sustainability Report...
1,LG생활건강,2013,20131023000180,https://kind.krx.co.kr/common/disclsviewer.do?...,https://kind.krx.co.kr/external/2013/10/23/000...,[2012 LG생활건강 CSR 보고서보고서 개요LG생활건강은 2010년 첫 지속가능...
2,LG유플러스,2013,20131002000021,https://kind.krx.co.kr/common/disclsviewer.do?...,https://kind.krx.co.kr/external/2013/10/02/000...,[LG U+2012 지속가능경영보고서About this Report본 보고서는 LG...
3,한화솔루션,2013,20131001000469,https://kind.krx.co.kr/common/disclsviewer.do?...,https://kind.krx.co.kr/external/2013/10/01/000...,[Sustainability Report본 지속가능성보고서는 FSC로부터 인증받은 ...
6,두산에너빌리티,2013,20130729000019,https://kind.krx.co.kr/common/disclsviewer.do?...,https://kind.krx.co.kr/external/2013/07/29/000...,[Building Your Tomorrow Today본사·창원 공장 경상남도 창원시...
...,...,...,...,...,...,...
490,한화솔루션,2023,20230530000639,https://kind.krx.co.kr/common/disclsviewer.do?...,https://kind.krx.co.kr/external/2023/06/23/000...,[Hanwha Solutions Sustainability Report 2023Su...
491,카카오페이,2023,20230530000636,https://kind.krx.co.kr/common/disclsviewer.do?...,https://kind.krx.co.kr/external/2023/05/30/000...,"[이로운 흐름, 유연한 금융2023ESGReport보고서 개요카카오페이는 당사의 서..."
492,카카오,2023,20230530000517,https://kind.krx.co.kr/common/disclsviewer.do?...,https://kind.krx.co.kr/external/2023/05/30/000...,[보고서 개요카카오는 더 나은 세상을 만들기 위한 회사의 약속과 책임 활동을 담은 ...
493,에스엘,2023,20230530000487,https://kind.krx.co.kr/common/disclsviewer.do?...,https://kind.krx.co.kr/external/2023/05/31/000...,[ENVIRONMENTAL SOCIAL GOVERNANCECORPORATION202...
